Thie notebook explores a Butler data repository and retrieve relevant eotest results. The `obs_lsst` package is required.

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import eups
assert eups.getSetupVersion("lsst_distrib")
print(eups.getSetupVersion("obs_lsst"))

In [ ]:
import lsst.afw.detection as afwDetection
from lsst.daf.persistence import Butler

import matplotlib.pyplot as plt
import numpy

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
# Look up what runs are available
runs = butler.queryMetadata('raw', ['run'], dataId={'imageType': 'FE55', 'testType': 'FE55'})
print("Available runs: ", runs)
# Look up what visits are available in this run
print("Available visits: ", butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FE55', 'testType': 'FE55', 'run': '7086' }))

# The old obs_lsst had a bug, so the dataId 'detector' was used instaed of 'detectorName'
# As of Nov 15, the new master of obs_lsst allow us to use `detectorName`
dId = {'visit': 258334666, 'detectorName': 'S00'}
raw = butler.get('raw', **dId)

# The following does not work at this moment; bug DM-16553 has been filed
#bias = butler.get('bias', visit=258334666, detectorName='S00')
# Use 'detector' before the bug is fixed
bias = butler.get('bias', visit=258334666, detector=0)

# One way to know which file this is:
#print(butler.get('raw_filename', **dId))

In [ ]:
# Alternatively, one can use butler subset feature 
# To match the same queryMetadata condition in the above cell
dataId = {'run': '7086', 'detectorName': 'S00', 'imageType': 'FE55', 'testType': 'FE55'}
subset = butler.subset("raw", **dataId)
print(len(subset))

n = 0
for ref in subset:
    print(ref.get("raw"))
    print(ref.dataId)
    # or more complicated operation can be done here
    n += 1
    if n > 3:
        break

In [ ]:
# Another way to get the detector name
raw.getDetector().getName()

In [ ]:
raw.getDetector().getSerial()

In [ ]:
# hard-coded gains in the camera definition inside obs_lsst
gains = [ccd.getGain() for ccd in raw.getDetector()]
print(gains)

To get EO test results from database, install the following packages to your area:

```
git clone https://github.com/lsst-camera-dh/eTraveler-clientAPI.git
git clone https://github.com/lsst-camera-dh/datacat-utilities.git 
cd datacat-utilities
git checkout python3
```

As of Nov 15, one need comment out all imports of `datacat` in `python/DataCatalog.py`; `datacat` is not needed to get the EO test results.


edit $HOME/notebooks/.user_setups with:

```
export PYTHONPATH=$PYTHONPATH:/path/to/eTraveler-clientAPI/python
export PYTHONPATH=$PYTHONPATH:/path/to/datacat-utilities/python
```

In [ ]:
from get_EO_analysis_results import get_EO_analysis_results
from get_EO_analysis_files import get_EO_analysis_files
from exploreFocalPlane import exploreFocalPlane
from exploreRaft import exploreRaft
from eTraveler.clientAPI.connection import Connection

In [ ]:
# Get EO analysis results access results from the eTraveler database
g = get_EO_analysis_results(db='Dev')

In [ ]:
# Some examples of getting eotest results
raft_list, data = g.get_tests(site_type="I&T-Raft", test_type="bright_columns", run ='5943D')  # get the data for I&T-Raft
res = g.get_results(test_type='bright_columns', data=data, device=raft_list[0])  # get the data for a raft

raft_list, data = g.get_tests(site_type="I&T-Raft", run ='5943D')
res = g.get_all_results(data=data, device=raft_list)
res.keys()

In [ ]:
# run numbers ending with 'D' is in the 'Dev' db
# run numbers ending without 'D' is in the 'Prod' db
g_prod = get_EO_analysis_results(db='Prod')
# Here we ask for the same run as the data in the Butler repo
raft_list, data = g_prod.get_tests(site_type="I&T-Raft", run ='7086')
res = g_prod.get_all_results(data=data, device=raft_list)

In [ ]:
res['gain'].keys()

Wait... The ccd names are different from those obtained from `exposure.getDetector().getSerial()`!

---> See ChrisW's notebook for more exploration!

In [ ]:
ccd_name = 'E2V-CCD250-260'
res['gain'][ccd_name]

In [ ]:
fig, axs = plt.subplots(1, 9, figsize=(20, 10), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0, wspace=0)
for i, ccd in enumerate(res['gain']):
    p = axs[i]
    p.set_title(ccd)
    p.errorbar(range(1,17),res['gain'][ccd],yerr=res['gain_error'][ccd],color='blue',label='Fe55 Gain', fmt='o')
    p.errorbar(range(1,17),res['ptc_gain'][ccd],yerr=res['ptc_gain_error'][ccd],color='red',label='PTC Gain', fmt='o')


from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='blue', lw=4),
                Line2D([0], [0], color='red', lw=4)]
fig.legend(custom_lines, ['Fe55 Gain', 'PTC Gain'])
fig.suptitle(str(dId), fontsize=12)
fig.savefig("plot.png")

### To make a mosaic plot using eotest

In [ ]:
import lsst.eotest.raft as raftTest

In [ ]:
import os
import glob

In [ ]:
visit = 258334666

In [ ]:
# Use file names
infiles = sorted(glob.glob(os.path.join("/project/bootcamp/repo_RTM-007/raw/7086/",
                                        "*"+str(visit)+"*")))

file_dict = {filename.split('-')[-2]: filename for filename in infiles}
print(file_dict)

In [ ]:
# Alternatively, use Butler
subset = butler.subset("raw", visit=visit)
assert(len(subset)==9)
file_dict = {ref.dataId['detectorName']: ref.get("raw_filename")[0][:-3] for ref in subset}

In [ ]:
raft_mosaic = raftTest.RaftMosaic(file_dict, bias_subtract=False)
raft_mosaic.plot(title='Test pattern')